# 1、练习groupby

In [2]:
import pandas as pd
import numpy as np
dict_obj = {'key1' : ['a', 'b', 'a', 'b',
                      'a', 'b', 'a', 'a'],
            'key2' : ['one', 'one', 'two', 'three',
                      'two', 'two', 'one', 'three'],
            'data1': np.random.randint(1, 10, 8),
            'data2': np.random.randint(1, 10, 8)}
df_obj = pd.DataFrame(dict_obj)
print(df_obj)

  key1   key2  data1  data2
0    a    one      5      3
1    b    one      6      2
2    a    two      2      7
3    b  three      6      9
4    a    two      5      5
5    b    two      7      8
6    a    one      6      6
7    a  three      1      1


In [3]:
# 按key1分组后，计算data1，data2的统计信息并附加到原始表格中，并添加表头前缀
k1_sum = df_obj.groupby('key1').mean(numeric_only=True).add_prefix('mean_')
print(k1_sum)

      mean_data1  mean_data2
key1                        
a       3.800000    4.400000
b       6.333333    6.333333


In [4]:
# 选出key1，data1，data2，再按照key1进行分组求均值，并添加表头前缀
k1_sum_tf = df_obj.loc[:,['key1','data1', 'data2']].groupby('key1').transform('mean').add_prefix('mean_')
print(k1_sum_tf)

   mean_data1  mean_data2
0    3.800000    4.400000
1    6.333333    6.333333
2    3.800000    4.400000
3    6.333333    6.333333
4    3.800000    4.400000
5    6.333333    6.333333
6    3.800000    4.400000
7    3.800000    4.400000


In [13]:
del df_obj['key2']
df_obj.groupby('key1').transform('mean')
#实现a组和b组，谁比平均分高，谁比平均分低
def diff_mean(s):
    """
        返回数据与均值的差值，s传入的是某一个分组
    """
    return s - s.mean()

print(df_obj.groupby('key1').transform(diff_mean))

      data1     data2
0  1.200000 -1.400000
1 -0.333333 -4.333333
2 -1.800000  2.600000
3 -0.333333  2.666667
4  1.200000  0.600000
5  0.666667  1.666667
6  2.200000  1.600000
7 -2.800000 -3.400000


# 2、练习merge

In [18]:
df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                        'data1' : np.random.randint(0,10,7)})
df_obj2 = pd.DataFrame({'key': ['a', 'b' ,'d'],
                        'data2' : np.random.randint(0,10,3)})

print(df_obj1)
print('-'*50)
print(df_obj2)

  key  data1
0   b      3
1   b      5
2   a      3
3   c      9
4   a      0
5   a      3
6   b      2
--------------------------------------------------
  key  data2
0   a      4
1   b      2
2   d      3


In [19]:
#默认连接使用相同的列名，连接方式是内连接
pd.merge(df_obj1, df_obj2)

,key,data1,data2
0,b,3,2
1,b,5,2
2,a,3,4
3,a,0,4
4,a,3,4
5,b,2,2


In [20]:
#左df和右df都拿索引连接
pd.merge(df_obj1, df_obj2,left_index=True,right_index=True)

,key_x,data1,key_y,data2
0,b,3,a,4
1,b,5,b,2
2,a,3,d,3


In [21]:
#左表和右表都拿key列来连接
pd.merge(df_obj1, df_obj2, on='key')

,key,data1,data2
0,b,3,2
1,b,5,2
2,a,3,4
3,a,0,4
4,a,3,4
5,b,2,2


In [25]:
# 更改列名
df_obj1 = df_obj1.rename(columns={'key':'key1'})
df_obj2 = df_obj2.rename(columns={'key':'key2'})
print(df_obj1)
print('-'*50)
print(df_obj2)

  key1  data1
0    b      3
1    b      5
2    a      3
3    c      9
4    a      0
5    a      3
6    b      2
--------------------------------------------------
  key2  data2
0    a      4
1    b      2
2    d      3


In [26]:
#左表以key1来连接，右表以key2来连接
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2')

,key1,data1,key2,data2
0,b,3,b,2
1,b,5,b,2
2,a,3,a,4
3,a,0,a,4
4,a,3,a,4
5,b,2,b,2


In [27]:
#全外连接
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='outer')

,key1,data1,key2,data2
0,a,3.0,a,4.0
1,a,0.0,a,4.0
2,a,3.0,a,4.0
3,b,3.0,b,2.0
4,b,5.0,b,2.0
5,b,2.0,b,2.0
6,c,9.0,NaN,NaN
7,NaN,NaN,d,3.0


In [28]:
#left等价于 left  join
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='left')

,key1,data1,key2,data2
0,b,3,b,2.0
1,b,5,b,2.0
2,a,3,a,4.0
3,c,9,NaN,NaN
4,a,0,a,4.0
5,a,3,a,4.0
6,b,2,b,2.0


In [29]:
# right等价于数据库的right join
pd.merge(df_obj1, df_obj2, left_on='key1', right_on='key2', how='right')

,key1,data1,key2,data2
0,a,3.0,a,4
1,a,0.0,a,4
2,a,3.0,a,4
3,b,3.0,b,2
4,b,5.0,b,2
5,b,2.0,b,2
6,NaN,NaN,d,3


In [30]:
# 处理重复列名
df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                        'data' : np.random.randint(0,10,7)})
df_obj2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                        'data' : np.random.randint(0,10,3)})
#给相同的数据列添加后缀
print(pd.merge(df_obj1, df_obj2, on='key', suffixes=('_left', '_right')))

  key  data_left  data_right
0   b          9           5
1   b          0           5
2   a          1           9
3   a          6           9
4   a          5           9
5   b          6           5


In [31]:
# 按索引连接
df_obj1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                        'data1' : np.random.randint(0,10,7)})
df_obj2 = pd.DataFrame({'data2' : np.random.randint(0,10,3)}, index=['a', 'b', 'd'])
print(df_obj1)
print(df_obj2)

  key  data1
0   b      9
1   b      6
2   a      0
3   c      6
4   a      8
5   a      1
6   b      2
   data2
a      6
b      1
d      9


In [32]:
print(pd.merge(df_obj1, df_obj2, left_on='key', right_index=True))

  key  data1  data2
0   b      9      1
1   b      6      1
2   a      0      6
4   a      8      6
5   a      1      6
6   b      2      1


In [33]:
pd.merge(df_obj2,df_obj1, left_index=True, right_on='key')

,data2,key,data1
2,6,a,0
4,6,a,8
5,6,a,1
0,1,b,9
1,1,b,6
6,1,b,2


# 3、练习重复值处理

In [35]:
df_obj = pd.DataFrame({'data1' : ['a'] * 4 + ['b'] * 4,
                       'data2' : np.random.randint(0, 4, 8)})
print(df_obj)

  data1  data2
0     a      3
1     a      3
2     a      2
3     a      1
4     b      2
5     b      1
6     b      3
7     b      1


In [36]:
print(df_obj.duplicated())

0    False
1     True
2    False
3    False
4    False
5    False
6    False
7     True
dtype: bool


In [37]:
df_obj[~df_obj.duplicated()]  #取出不重复行

,data1,data2
0,a,3
2,a,2
3,a,1
4,b,2
5,b,1
6,b,3


In [38]:
#按照某一列去重
print(df_obj.duplicated('data2'))

0    False
1     True
2    False
3    False
4     True
5     True
6     True
7     True
dtype: bool


In [39]:
df_obj1 = pd.DataFrame({'data1' :[np.nan] * 4,
                       'data2' :list('1235')})
df_obj1

,data1,data2
0,NaN,1
1,NaN,2
2,NaN,3
3,NaN,5


In [40]:
#在pd的duplicated认为空值和空值相等的
df_obj1.duplicated('data1')

0    False
1     True
2     True
3     True
dtype: bool

In [41]:
print(df_obj.drop_duplicates())  #删除重复行,没有改变原有对象，而是返回了一个新对象

  data1  data2
0     a      3
2     a      2
3     a      1
4     b      2
5     b      1
6     b      3


In [43]:
#map与applymap一样，但是map只能用于series，applymap只能用于df
ser_obj = pd.Series(np.random.randint(0,10,10))  #series 用map
print(ser_obj)
print('-'*50)
print(ser_obj.map(lambda x : x ** 2))

0    1
1    1
2    5
3    7
4    5
5    9
6    4
7    6
8    0
9    7
dtype: int32
--------------------------------------------------
0     1
1     1
2    25
3    49
4    25
5    81
6    16
7    36
8     0
9    49
dtype: int64


In [44]:
#异常值手动替换
ser_obj=pd.Series(np.arange(10),index=range(3,13))
# 单个值替换单个值
print(ser_obj.replace(1, -100))
print('-' * 20)
# 多个值替换一个值
print(ser_obj.replace(range(6,9), -100))
print('-' * 20)
# 多个值替换多个值
print(ser_obj.replace([4, 7], [-100, -200]))

3       0
4    -100
5       2
6       3
7       4
8       5
9       6
10      7
11      8
12      9
dtype: int64
--------------------
3       0
4       1
5       2
6       3
7       4
8       5
9    -100
10   -100
11   -100
12      9
dtype: int64
--------------------
3       0
4       1
5       2
6       3
7    -100
8       5
9       6
10   -200
11      8
12      9
dtype: int64


In [47]:
df = pd.DataFrame({'A': [0, 1, 2, 3, 4],
                   'B': [5, 6, 7, 8, 9],
                   'C': ['a', 'b', 'ac', 'd', 'e']})
print(df)

   A  B   C
0  0  5   a
1  1  6   b
2  2  7  ac
3  3  8   d
4  4  9   e


In [50]:
#正则表达式替换
df.replace(to_replace=r'^a', value=100, regex=True) #a开头的字符串替换为100

,A,B,C
0,0,5,100
1,1,6,b
2,2,7,100
3,3,8,d
4,4,9,e


In [53]:
df # 原对象未被改变

,A,B,C
0,0,5,a
1,1,6,b
2,2,7,ac
3,3,8,d
4,4,9,e
